In [1]:
import models
from models import combine_bench_prior, load_priorband_data,add_rel_ranks,rename_algos,create_incumbent
import pandas as pd
import numpy as np

loss="value";algorithm="algorithm";benchmark="bench_prior";seed="seed"
algos=["pb_mutation_dynamic_geometric-default-at-target","random_search","hyperband"]
fs=[1,5,12]
f_space=np.linspace(1,max(fs),max(fs)).tolist()
benchmarks=["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]
label_dict={'random_search': 'RS', 'hyperband': 'HB', 'pb_mutation_dynamic_geometric-default-at-target': 'PB', 'jahs_cifar10': 'JAHS-C10', 'jahs_colorectal_histology': 'JAHS-CH', 'jahs_fashion_mnist': 'JAHS-FM', 'lcbench-126026': 'LC-126026', 'lcbench-167190': 'LC-167190', 'lcbench-168330': 'LC-168330', 'lcbench-168910': 'LC-168910', 'lcbench-189906': 'LC-189906', 'cifar100_wideresnet_2048': 'PD1-Cifar100', 'imagenet_resnet_512': 'PD1-ImageNet', 'lm1b_transformer_2048': 'PD1-LM1B', 'translatewmt_xformer_64': 'PD1-WMT',"random_search_prior":"RS+Prior","bo":"BO","pibo":"PiBO","bohb":"BOHB","priorband_bo":"PriorBand+BO"}

data=load_priorband_data()
data=data.loc[(data[algorithm].isin(algos))&(data["benchmark"].isin(benchmarks))]
data["bench_prior"] = data.apply(combine_bench_prior, axis=1)
data.drop(columns=["benchmark","prior"],inplace=True)
benchmarks=data[benchmark].unique()
dataset=create_incumbent(data,fs,f_space,benchmarks,algos,benchmark,algorithm)

for max_f in dataset.keys():
    print(f"⚙️ F {max_f}: Adding relative ranks             ",end="\r", flush=True)
    data=dataset[max_f]
    data["rel_rank"]=data.apply(add_rel_ranks,data=data,benchmark=benchmark,axis=1)
    print(f"⚙️ F {max_f}: Renaming algorithms             ",end="\r", flush=True)
    data[algorithm]=data.apply(rename_algos,algo_dict=label_dict,axis=1)
    dataset[max_f]=data
print("✅ Dataset loaded                   ",end="\r", flush=True)


✅ Loading data done     


In [2]:
for fidelity in dataset.keys():
    data=dataset[fidelity]
    print(f"Fidelity: {fidelity}")
    m0_bar=models.model(formula=f"{loss}~{algorithm}+(1|{seed})",system_id=algorithm,data=data)
    m0=models.model(formula=f"{loss}~{algorithm}+(1|{seed})+(1|{benchmark})",system_id=algorithm,data=data)
    m1=models.model(formula=f"{loss}~{algorithm}+(1|{seed})+{benchmark}",system_id=algorithm,data=data)

    print("Task 1️⃣")
    models.glrt(m1,m0_bar,names=["M1","M0'"])
    print("=> Benchmarks are important in determining algorithms behaviour \n")
    print("Task 2️⃣")
    models.glrt(m1,m0,names=["M1","M0"])
    print("=> And they should be a fixed factor")

Fidelity: 1
boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

Task 1️⃣
M1 (-22772.1) >> M0' (-25333.0)
Chi-Square: 5121.804437749117, P-Value: 0.0
=> Benchmarks are important in determining algorithms behaviour 

Task 2️⃣
M1 (-22772.1) >> M0 (-22898.22)
Chi-Square: 252.24155058001634, P-Value: 0.0
=> And they should be a fixed factor
Fidelity: 5
boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

Task 1️⃣
M1 (-99007.98) >> M0' (-106900.43)
Chi-Square: 15784.885718914331, P-Value: 0.0
=> Benchmarks are important in determining algorithms behaviour 

Task 2️⃣
M1 (-99007.98) >> M0 (-99153.02)
Chi-Square: 290.07407930042245, P-Value: 0.0
=> And they should be a fixed factor
Fidelity: 12
boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (

In [3]:
# for fidelity in dataset.keys():
#     data=dataset[fidelity]
#     print(f"Fidelity: {fidelity}")
#     m_s_1b=models.model(formula=f"{loss}~{algorithm}+{seed}+(1|{benchmark})",system_id=algorithm,data=data)
#     m_1b=models.model(formula=f"{loss}~{algorithm}+(1|{benchmark})",system_id=algorithm,data=data)
#     models.glrt(m_s_1b,m0,names=["m_s_1b","M0"])
#     models.glrt(m_s_1b,m_1b,names=["m_s_1b","m_1b"])
#     models.glrt(m_s_1b,m1,names=["m_s_1b","M1"])
#     models.glrt(m1,m_1b,names=["M1","m_1b"])

In [4]:
for fidelity in dataset.keys():
    data=dataset[fidelity]
    print(f"Fidelity: {fidelity}")
    print("Task 3️⃣ (Relative Ranks)")
    m3_0_bar=models.model(formula=f"{loss}~{algorithm}+(1|{seed})",system_id=algorithm,data=data)
    m3_0=models.model(formula=f"{loss}~{algorithm}+(1|{seed})+(1|{benchmark})",system_id=algorithm,data=data)
    m3_1=models.model(formula=f"{loss}~{algorithm}+(1|{seed})+{benchmark}",system_id=algorithm,data=data)

    models.glrt(m3_1,m3_0_bar,names=["M1-rel","M0'-rel"])
    print("=> Benchmarks are important in determining algorithms behaviour \n")
    models.glrt(m3_1,m3_0,names=["M1-rel","M0-rel"])
    print("=> And they should be a fixed factor\n")
    print("Task 4️⃣")
    m2=models.model(formula=f"{loss}~{algorithm}+{benchmark}+{algorithm}:{benchmark}+(1|{seed})",system_id=algorithm,data=data)
    models.glrt(m1,m2,names=["M1","M2"])
    print("=> Interaction effect is significance \n")


Fidelity: 1
Task 3️⃣ (Relative Ranks)
boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

M1-rel (-22772.1) >> M0'-rel (-25333.0)
Chi-Square: 5121.804437749117, P-Value: 0.0
=> Benchmarks are important in determining algorithms behaviour 

M1-rel (-22772.1) >> M0-rel (-22898.22)
Chi-Square: 252.24155058001634, P-Value: 0.0
=> And they should be a fixed factor

Task 4️⃣
M1 (-210689.37) << M2 (-14571.72)
Chi-Square: 392235.30086470215, P-Value: 0.0
=> Interaction effect is significance 

Fidelity: 5
Task 3️⃣ (Relative Ranks)
boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

M1-rel (-99007.98) >> M0'-rel (-106900.43)
Chi-Square: 15784.885718914331, P-Value: 0.0
=> Benchmarks are important in determining algorithms behaviour 

M1-rel (-99007.98) >> M0-rel (-99153.02)
Chi-Square: 290.074079300422